In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 4つのサブプロットを作成
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=(
        'Stock Returns', 
        'Copula Correlation Parameter (ρ)', 
        'Anomaly Scores',
        'Detected Anomalies'
    ),
    shared_xaxes=True,
    vertical_spacing=0.08,
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": False}]]
)

# 1. 株価収益率
fig.add_trace(
    go.Scatter(x=results_df['Date'], y=results_df['Nikkei_Return'], 
               name='Nikkei Returns', line=dict(color='red'), opacity=0.7),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=results_df['Date'], y=results_df['SP500_Return'], 
               name='S&P500 Returns', line=dict(color='blue'), opacity=0.7),
    row=1, col=1
)

# 2. コピュラ相関パラメータ
fig.add_trace(
    go.Scatter(x=results_df['Date'], y=results_df['Fitted_Rho'], 
               name='Copula ρ', line=dict(color='green'), opacity=0.8),
    row=2, col=1
)

# 3. 異常スコア
fig.add_trace(
    go.Scatter(x=results_df['Date'], y=results_df['Anomaly_Score'], 
               name='Anomaly Score', line=dict(color='purple'), opacity=0.8),
    row=3, col=1
)

# 閾値線を追加
fig.add_hline(y=threshold, line_dash="dash", line_color="red", 
              annotation_text=f"Threshold ({threshold:.2f})", row=3, col=1)

# 4. 検知された異常
anomaly_y = [1] * len(anomaly_dates)
fig.add_trace(
    go.Scatter(x=anomaly_dates, y=anomaly_y,
               mode='markers', name='Detected Anomalies', 
               marker=dict(color='red', size=8, symbol='diamond')),
    row=4, col=1
)

# 主要危機期間をハイライト
colors = ['rgba(255,0,0,0.1)', 'rgba(0,255,0,0.1)', 'rgba(0,0,255,0.1)', 'rgba(255,255,0,0.1)']
for i, (crisis_name, (start, end)) in enumerate(crisis_periods.items()):
    for row in range(1, 5):
        fig.add_vrect(
            x0=start, x1=end,
            fillcolor=colors[i % len(colors)],
            opacity=0.3,
            layer="below",
            line_width=0,
            row=row, col=1
        )

# レイアウト調整
fig.update_layout(
    height=1000,
    title_text="Copula-based Financial Anomaly Detection System",
    showlegend=True,
    hovermode='x unified'
)

fig.update_xaxes(title_text="Date", row=4, col=1)
fig.update_yaxes(title_text="Returns", row=1, col=1)
fig.update_yaxes(title_text="Correlation", row=2, col=1)
fig.update_yaxes(title_text="Score", row=3, col=1)
fig.update_yaxes(title_text="Anomaly", row=4, col=1)

fig.show()

# 散布図での可視化
fig2 = go.Figure()

# 通常データ
normal_mask = ~anomalies
fig2.add_trace(go.Scatter(
    x=results_df[normal_mask]['Nikkei_Return'],
    y=results_df[normal_mask]['SP500_Return'],
    mode='markers',
    name='Normal',
    marker=dict(color='blue', size=4, opacity=0.6)
))

# 異常データ
fig2.add_trace(go.Scatter(
    x=results_df[anomalies]['Nikkei_Return'],
    y=results_df[anomalies]['SP500_Return'],
    mode='markers',
    name='Anomaly',
    marker=dict(color='red', size=8, opacity=0.8)
))

fig2.update_layout(
    title="Returns Scatter Plot with Detected Anomalies",
    xaxis_title="Nikkei Returns",
    yaxis_title="S&P500 Returns",
    hovermode='closest'
)

fig2.show()

In [ ]:
# 従来の相関ベース異常検知
rolling_window = 30
rolling_corr = results_df['Nikkei_Return'].rolling(rolling_window).corr(results_df['SP500_Return'])
corr_change = rolling_corr.diff().abs()
corr_threshold = corr_change.quantile(0.95)
corr_anomalies = corr_change > corr_threshold

# 日付を文字列に変換して比較
copula_anomaly_dates = set(results_df[anomalies]['Date'].dt.strftime('%Y-%m-%d'))
corr_anomaly_dates = set(results_df[corr_anomalies & ~corr_change.isna()]['Date'].dt.strftime('%Y-%m-%d'))

print("=== コピュラベース vs 相関ベース異常検知の比較 ===")
print(f"コピュラのみ検知: {len(copula_anomaly_dates - corr_anomaly_dates)}件")
print(f"相関のみ検知: {len(corr_anomaly_dates - copula_anomaly_dates)}件")
print(f"両方で検知: {len(copula_anomaly_dates & corr_anomaly_dates)}件")
print(f"総検知数（和集合）: {len(copula_anomaly_dates | corr_anomaly_dates)}件")

# コピュラ独自検知の分析
copula_unique = copula_anomaly_dates - corr_anomaly_dates
print(f"\n=== コピュラが独自に検知した異常の例 ===")
for date in list(copula_unique)[:5]:
    date_data = results_df[results_df['Date'].dt.strftime('%Y-%m-%d') == date]
    if not date_data.empty:
        row = date_data.iloc[0]
        print(f"{date}: "
              f"Score={row['Anomaly_Score']:.3f}, "
              f"Nikkei={row['Nikkei_Return']:.3%}, "
              f"SP500={row['SP500_Return']:.3%}, "
              f"ρ={row['Fitted_Rho']:.3f}")

# 比較の可視化
comparison_fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Correlation Changes', 'Copula Anomaly Scores', 'Detection Comparison'),
    shared_xaxes=True
)

# 相関変化
comparison_fig.add_trace(
    go.Scatter(x=results_df['Date'], y=corr_change, 
               name='Correlation Change', line=dict(color='orange')),
    row=1, col=1
)
comparison_fig.add_hline(y=corr_threshold, line_dash="dash", line_color="orange", row=1, col=1)

# コピュラ異常スコア
comparison_fig.add_trace(
    go.Scatter(x=results_df['Date'], y=results_df['Anomaly_Score'], 
               name='Copula Score', line=dict(color='purple')),
    row=2, col=1
)
comparison_fig.add_hline(y=threshold, line_dash="dash", line_color="purple", row=2, col=1)

# 検知比較
corr_only_dates = results_df[corr_anomalies & ~corr_change.isna()]['Date']
copula_only_dates = anomaly_dates
both_dates = results_df[anomalies & corr_anomalies]['Date']

comparison_fig.add_trace(
    go.Scatter(x=corr_only_dates, y=[1]*len(corr_only_dates),
               mode='markers', name='Correlation Only', 
               marker=dict(color='orange', size=6)),
    row=3, col=1
)

comparison_fig.add_trace(
    go.Scatter(x=copula_only_dates, y=[2]*len(copula_only_dates),
               mode='markers', name='Copula Only', 
               marker=dict(color='purple', size=6)),
    row=3, col=1
)

comparison_fig.add_trace(
    go.Scatter(x=both_dates, y=[3]*len(both_dates),
               mode='markers', name='Both Methods', 
               marker=dict(color='red', size=8)),
    row=3, col=1
)

comparison_fig.update_layout(height=800, title_text="Copula vs Correlation-based Anomaly Detection")
comparison_fig.show()